Logistic Regression

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.model_selection import GridSearchCV
import datetime
from sklearn.preprocessing import OneHotEncoder
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [23]:
df_logistic = pd.read_csv('../../raw_data/latlong.csv')


In [24]:
df_logistic = df_logistic.dropna()

In [25]:
df_logistic = df_logistic.sample(50000)

In [26]:
df_logistic.shape

(50000, 14)

In [43]:
df_logistic


,MONTH,DAY,DAY_OF_WEEK,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLED,ORIGIN_LONGITUDE,...,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN
4078,3,8,7,ORD,SEA,275.0,1721,2210,0,-87.90446,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86350,2,10,2,TUS,SLC,117.0,601,1254,0,-110.94103,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
362530,6,12,5,MDW,MSP,85.0,349,1425,0,-87.75242,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
245783,7,14,2,SJU,ATL,232.0,1547,1942,0,-66.00183,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
119814,8,19,3,SEA,PHX,166.0,1107,1111,0,-122.30931,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319037,5,5,2,XNA,IAH,101.0,438,2020,0,-94.30681,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
405157,7,21,2,ATL,DCA,110.0,547,1950,0,-84.42694,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
282799,2,1,7,DCA,DTW,115.0,405,1450,1,-77.03772,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126412,8,16,7,ATL,PHX,232.0,1587,1237,0,-84.42694,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
X = df_logistic.drop(columns = ['AIRLINE','CANCELLED','DESTINATION_AIRPORT','ORIGIN_AIRPORT'])
y = df_logistic['CANCELLED']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)



In [28]:
#Scaling

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled =scaler.transform(X_test)

In [29]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder(sparse = False) 

# Fit encod
ohe.fit(df_logistic[['AIRLINE']]) 

# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")

The categories detected by the OneHotEncoder are [array(['AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'HA', 'MQ', 'NK', 'OO', 'UA',
       'US', 'VX', 'WN'], dtype=object)]


/home/ignacio/.pyenv/versions/3.10.6/envs/flight-predictor/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [30]:
# Display the generated names
print(f"The column names for the encoded values are {ohe.get_feature_names_out()}")

# Transform the current "Street" column
df_logistic[ohe.get_feature_names_out()] = ohe.transform(df_logistic[['AIRLINE']])

# Drop the column "Street" which has been encoded
df_logistic.drop(columns = ["AIRLINE"], inplace = True)

The column names for the encoded values are ['AIRLINE_AA' 'AIRLINE_AS' 'AIRLINE_B6' 'AIRLINE_DL' 'AIRLINE_EV'
 'AIRLINE_F9' 'AIRLINE_HA' 'AIRLINE_MQ' 'AIRLINE_NK' 'AIRLINE_OO'
 'AIRLINE_UA' 'AIRLINE_US' 'AIRLINE_VX' 'AIRLINE_WN']


In [31]:
# Show the dataset
df_logistic.head(3)

,MONTH,DAY,DAY_OF_WEEK,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLED,ORIGIN_LONGITUDE,...,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN
4078,3,8,7,ORD,SEA,275.0,1721,2210,0,-87.90446,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86350,2,10,2,TUS,SLC,117.0,601,1254,0,-110.94103,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
362530,6,12,5,MDW,MSP,85.0,349,1425,0,-87.75242,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [32]:
model = LogisticRegression()



In [33]:
model.fit(X_train_scaled, y_train)

LogisticRegression()

In [34]:
y_pred = model.predict(X_test_scaled)

In [35]:
confusion_matrix(y_test, y_pred)

array([[13141,     0],
       [ 1859,     0]])

In [36]:
cv_results = cross_validate(model, X, y, cv=5, scoring=['precision'])
pd.DataFrame(cv_results)

/home/ignacio/.pyenv/versions/3.10.6/envs/flight-predictor/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ignacio/.pyenv/versions/3.10.6/envs/flight-predictor/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ignacio/.pyenv/versions/3.10.6/envs/flight-predictor/lib/python3.10/site-packages/sklearn/linear

,fit_time,score_time,test_precision
0,2.929322,0.075900,0.0
1,3.331382,0.082691,0.0
2,2.799906,0.043125,0.0
3,3.132212,0.028763,0.0
4,1.538945,0.068984,0.0


In [37]:
logistic_score = cv_results['test_precision'].mean()
logistic_score

0.0

In [38]:
param_grid = {
    'C': [0.01, 0.1, 1], 
    'penalty': ['l1',' l2', 'elasticnet', 'none'],
    'max_iter': [100,500,1000],
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
    }

In [39]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, verbose=0, cv=5, n_jobs=-1)

In [40]:
grid_search.fit(X_train_scaled, y_train);

/home/ignacio/.pyenv/versions/3.10.6/envs/flight-predictor/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/ignacio/.pyenv/versions/3.10.6/envs/flight-predictor/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/ignacio/.pyenv/versions/3.10.6/envs/flight-predictor/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/ignacio/.pyenv/versions/3.10.6/envs/flight-predictor/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


In [41]:
grid_search.best_estimator_

LogisticRegression(C=0.01, penalty='l1', solver='liblinear')

In [42]:
grid_search.best_score_

0.8826857142857143